In [1]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvid

In [2]:
import os
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
import torch
import evaluate

def replace_tags(input_filepath, output_filepath):
    """
    Читає файл, замінює теги 'B-PER' та 'I-PER' на 'B-PERS' та 'I-PERS'
    відповідно, та зберігає результат у новий файл.
    """
    try:
        with open(input_filepath, 'r', encoding='utf-8') as infile:
            content = infile.read()
    except FileNotFoundError:
        print(f"Помилка: Файл не знайдено за шляхом '{input_filepath}'.")
        return False

    updated_content = content.replace(" B-PERS", " B-PER").replace(" I-PERS", " I-PER")
    
    with open(output_filepath, 'w', encoding='utf-8') as outfile:
        outfile.write(updated_content)

    print(f"Заміну тегів завершено. Оновлений файл збережено за шляхом: {output_filepath}")
    return True
    
def process_iob_file(filepath):
    """
    Читає файл IOB та повертає список словників, де кожен словник представляє речення.
    Фільтрує дані, щоб залишити лише теги ORG та PERS.
    """
    valid_tags = {'B-ORG', 'I-ORG', 'B-PER', 'I-PER', 'O'}
    data = []
    current_tokens = []
    current_tags = []
    
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                if current_tokens:
                    data.append({"tokens": current_tokens, "ner_tags": current_tags})
                    current_tokens = []
                    current_tags = []
            else:
                token, tag = line.split()
                if tag not in valid_tags:
                    tag = 'O'
                current_tokens.append(token)
                current_tags.append(tag)

    if current_tokens:
        data.append({"tokens": current_tokens, "ner_tags": current_tags})
        
    return data

train_data = process_iob_file("/kaggle/input/data-ukr/train.iob")
test_data = process_iob_file("/kaggle/input/data-ukr/test.iob")
original_train_path = "/kaggle/input/data-ukr/train.iob"
original_test_path = "/kaggle/input/data-ukr/test.iob"
updated_train_path = "train_pers.iob"
updated_test_path = "test_pers.iob"

if replace_tags(original_train_path, updated_train_path) and replace_tags(original_test_path, updated_test_path):
    train_data = process_iob_file(updated_train_path)
    test_data = process_iob_file(updated_test_path)

2025-08-28 20:09:28.582135: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756411768.775926      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756411768.834786      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Заміну тегів завершено. Оновлений файл збережено за шляхом: train_pers.iob
Заміну тегів завершено. Оновлений файл збережено за шляхом: test_pers.iob


In [3]:
train_dataset = Dataset.from_list(train_data)
test_dataset = Dataset.from_list(test_data)

print(f"Original dataset size: {len(train_data)} training sentences, {len(test_data)} test sentences.")


Original dataset size: 11161 training sentences, 5087 test sentences.


In [4]:
model_checkpoint = "youscan/ukr-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

label_list = ['O', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER']
label2id = {l: i for i, l in enumerate(label_list)}
id2label = {i: l for i, l in enumerate(label_list)}

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint, 
    num_labels=len(label_list), 
    id2label=id2label, 
    label2id=label2id
)

tokenizer_config.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/507M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at youscan/ukr-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
train_dataset

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 11161
})

In [6]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
    )
    labels_list = []
    for i, labels in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[labels[word_idx]])
            else:
                label_ids.append(label2id[labels[word_idx]])
            previous_word_idx = word_idx
        labels_list.append(label_ids)
    
    tokenized_inputs["labels"] = labels_list
    return tokenized_inputs


tokenized_train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/11161 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/507M [00:00<?, ?B/s]

Map:   0%|          | 0/5087 [00:00<?, ? examples/s]

In [7]:
tokenized_train_dataset

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 11161
})

In [8]:
print(tokenized_train_dataset[0])

{'tokens': ['Дорогою', 'зупинялися', 'в', 'селах', ',', 'старцювали', '.', 'Устимко', 'приспівував', 'дідам', '.', 'Амфібрахій', 'шукав', 'підробіток', '.'], 'ner_tags': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O'], 'input_ids': [0, 42549, 500, 3326, 23574, 281, 15360, 1393, 2112, 792, 710, 1473, 407, 7203, 427, 390, 9608, 4087, 15664, 460, 1473, 12737, 724, 303, 741, 386, 34353, 426, 1770, 372, 1473, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, -100]}


In [9]:
from transformers import DataCollatorForTokenClassification

In [10]:
pip install seqeval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=0c956780d6b346376ccc98d346e7e2138ae343d0b998128099bff6099a9b3f96
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
Note: you may need to restart the kernel to use updated packages.


In [11]:
import numpy as np
from seqeval.metrics import precision_score, recall_score, f1_score, accuracy_score

In [12]:
metric = evaluate.load("seqeval")
data_collator = DataCollatorForTokenClassification(tokenizer)

def compute_metrics(p):
    preds_logits, labels = p
    preds = np.argmax(preds_logits, axis=2)

    id2label = model.config.id2label

    true_labels = [
        [id2label[l] for l in label_seq if l != -100]
        for label_seq in labels
    ]
    true_preds = [
        [id2label[p] for p, l in zip(pred_seq, label_seq) if l != -100]
        for pred_seq, label_seq in zip(preds, labels)
    ]

    precision = precision_score(true_labels, true_preds)
    recall = recall_score(true_labels, true_preds)
    f1 = f1_score(true_labels, true_preds)
    acc = accuracy_score(true_labels, true_preds)

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "accuracy": acc,
    }

In [13]:
model_output_dir = "ukr_ner_model"
training_args = TrainingArguments(
    output_dir=model_output_dir,
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    logging_dir='./logs',
    logging_steps=100,
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)


In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


/tmp/ipykernel_36/3986238953.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [15]:
print("Starting training...")
res = trainer.train()


Starting training...


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.039700,0.034813,0.884865,0.893426,0.889125,0.987793
2,0.016200,0.034414,0.888564,0.907655,0.898008,0.989126
3,0.007100,0.039309,0.892817,0.909078,0.900874,0.989032


In [16]:
output_dir = "./ukr_fine_tuned_model"

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model and tokenizer saved to {output_dir}")

Model and tokenizer saved to ./ukr_fine_tuned_model


In [17]:
metrics = trainer.evaluate(tokenized_test_dataset)
metrics

{'eval_loss': 0.03441426157951355,
 'eval_precision': 0.8885638668338208,
 'eval_recall': 0.907655093910074,
 'eval_f1': 0.898008024213416,
 'eval_accuracy': 0.9891257099802945,
 'eval_runtime': 14.1173,
 'eval_samples_per_second': 360.337,
 'eval_steps_per_second': 22.525,
 'epoch': 3.0}

In [18]:
import shutil

In [19]:
shutil.make_archive('ukr_ner_model', 'zip', './ukr_ner_model')

# 2. Download it
from IPython.display import FileLink
FileLink(r'ukr_ner_model.zip')

/kaggle/working/ukr_ner_model.zip